# Self-Supervision

- Train a deep network using the self-supervised learning framework.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%cd ..

D:\GitHub\eeg_analysis


In [2]:
# Load some packages
import hydra
from omegaconf import OmegaConf
import wandb
import pprint

# custom package
from run_train import check_device_env
from run_mae_train import prepare_and_run_ssl_train

D:\GitHub\eeg_analysis\run_mae_train.py:70: UserWarning:


The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1



---

## Specify the dataset, model, and train setting

In [3]:
project = 'caueeg-mae'
data_cfg_file = 'caueeg-abnormal'
train_cfg_file = 'tiny-train'
ssl_cfg_file = '2D-MAE-B'
device = 'cuda:0'

---

## Initializing configurations using Hydra

In [7]:
with hydra.initialize(config_path='../config'):
    add_configs = [f"data={data_cfg_file}", 
                   f"++data.seq_length=4095",
                   f"++data.stft_params.n_fft=127",
                   f"++data.input_norm=datapoint",
                   f"++data.awgn=0.003",
                   f"++data.mgn=0.003",
                   f"++data.crop_multiple=4",
                   f"++data.resample=100",   ###################################################
                   f"train={train_cfg_file}",
                   f"+train.device={device}", 
                   f"+train.project={project}",
                   f"++train.lr_scheduler_type=cosine_decay_with_warmup_half",
                   f"++train.base_lr=1e-3",
                   f"++train.total_samples=1e+6",  ###############################################
                   f"++train.save_model=True",  ###############################################
                   f"++train.search_lr=False",   ###############################################
                   f"++train.mixed_precision=True",
                   f"ssl={ssl_cfg_file}",
                   f"++ssl.patch_size=8",
                   f"++ssl.use_age=no",
                   f"++ssl.norm_pix_loss=True",
                   f"++ssl.mask_ratio=0.75",
                  ]
    
    cfg = hydra.compose(config_name='default', overrides=add_configs)
    
config = {**OmegaConf.to_container(cfg.data), 
          **OmegaConf.to_container(cfg.train),
          **OmegaConf.to_container(cfg.ssl)}

check_device_env(config)
pprint.pprint(config)

{'EKG': 'O',
 '_target_': 'models.ssl.mae_2d.mae_2d_pre_b_e768_d512',
 'activation': 'gelu',
 'awgn': 0.003,
 'awgn_age': 0.001,
 'base_lr': 0.001,
 'criterion': 'cross-entropy',
 'crop_multiple': 4,
 'crop_timing_analysis': False,
 'dataset_name': 'caueeg-dataset',
 'dataset_path': 'local/dataset/caueeg-dataset/',
 'ddp': False,
 'device': device(type='cuda', index=0),
 'draw_result': True,
 'file_format': 'memmap',
 'input_norm': 'datapoint',
 'latency': 2000,
 'load_event': False,
 'lr_scheduler_type': 'cosine_decay_with_warmup_half',
 'mask_ratio': 0.75,
 'mgn': 0.003,
 'minibatch': 256,
 'minibatch_3090': 256,
 'mixed_precision': True,
 'mixup': 0.0,
 'model': '2D-MAE-B',
 'norm_pix_loss': True,
 'num_history': 5,
 'patch_size': 8,
 'photic': 'X',
 'project': 'caueeg-mae',
 'resample': 100,
 'run_mode': 'train',
 'sampling_rate': 200,
 'save_model': True,
 'search_lr': False,
 'search_multiplier': 1.0,
 'seed': 0,
 'seq_length': 4095,
 'signal_length_limit': 10000000,
 'stft_param

C:\Users\Minjae\AppData\Local\Temp\ipykernel_26512\3320556640.py:1: UserWarning:


The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1



## Train

In [8]:
%%time
prepare_and_run_ssl_train(rank=None, world_size=None, config=config)


******************************    Configurations for Train    ******************************

{'EKG': 'O',
 '_target_': 'models.ssl.mae_2d.mae_2d_pre_b_e768_d512',
 'activation': 'gelu',
 'age_mean': tensor([71.3079]),
 'age_std': tensor([9.7947]),
 'awgn': 0.003,
 'awgn_age': 0.001,
 'base_lr': 0.001,
 'class_label_to_name': ['Normal', 'Abnormal'],
 'class_name_to_label': {'Abnormal': 1, 'Normal': 0},
 'criterion': 'cross-entropy',
 'crop_length': 4095,
 'crop_multiple': 4,
 'crop_timing_analysis': False,
 'dataset_name': 'CAUEEG dataset',
 'dataset_path': 'local/dataset/caueeg-dataset/',
 'ddp': False,
 'device': device(type='cuda', index=0),
 'draw_result': True,
 'file_format': 'memmap',
 'in_channels': 40,
 'input_norm': 'datapoint',
 'latency': 2000,
 'load_event': False,
 'lr_scheduler_type': 'cosine_decay_with_warmup_half',
 'mask_ratio': 0.75,
 'mgn': 0.003,
 'minibatch': 256,
 'minibatch_3090': 256,
 'mixed_precision': True,
 'mixup': 0.0,
 'model': '2D-MAE-B',
 'multi_batch